In [105]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn import decomposition
from scipy import stats
from sklearn import cluster
from vincenty import vincenty
from h3 import h3
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import folium
import branca.colormap as cm
from geojson.feature import *
import json
from IPython.display import Image, display
import calendar
from tqdm import tqdm

In [106]:
%time

df_list = [] # list to hold the batch dataframe

TRAIN_PATH = 'C:/Users/umang/Downloads/NYPD_Complaint_Data_Current__Year_To_Date_.csv'

# Peep at the training file header
df_tmp = pd.read_csv(TRAIN_PATH)
print(df_tmp.head())

df_tmp.info()

# Set columns to most suitable type to optimize for memory usage



traintypes = {'Latitude': 'float32',
              'Longitude': 'float32'}
cols = traintypes.keys()

#cols = list(traintypes.keys())
chunksize = 1_000_000 

for df_chunk in tqdm(pd.read_csv(TRAIN_PATH,  dtype=traintypes, chunksize=chunksize, usecols=cols)):#, parse_dates=[0,1])): #usecols=cols,
     
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
    #df_chunk['tpep_pickup_datetime'] = df_chunk['tpep_pickup_datetime'].str.slice(0, 16)
    #df_chunk['tpep_pickup_datetime'] = pd.to_datetime(df_chunk['tpep_pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    
    # Can process each chunk of dataframe here
    # clean_data(), feature_engineer(),fit()
    
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk)


Wall time: 0 ns
   CMPLNT_NUM  ADDR_PCT_CD BORO_NM CMPLNT_FR_DT CMPLNT_FR_TM CMPLNT_TO_DT  \
0   885776788           66     NaN   12/23/2020     19:50:00          NaN   
1   350637195           77     NaN   12/21/2020     01:10:00          NaN   
2   347843168           43   BRONX   11/22/2020     22:00:00          NaN   
3   197941396           47     NaN   11/22/2020     09:50:00          NaN   
4   298404927           25     NaN   11/21/2020     15:38:00          NaN   

  CMPLNT_TO_TM CRM_ATPT_CPTD_CD HADEVELOPT  HOUSING_PSA  ...  \
0          NaN        COMPLETED        NaN          NaN  ...   
1          NaN        COMPLETED        NaN          NaN  ...   
2          NaN        COMPLETED        NaN          NaN  ...   
3          NaN        COMPLETED        NaN          NaN  ...   
4          NaN        COMPLETED        NaN          NaN  ...   

   TRANSIT_DISTRICT VIC_AGE_GROUP        VIC_RACE VIC_SEX X_COORD_CD  \
0               NaN         18-24           BLACK       M     98

0it [00:00, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413412 entries, 0 to 413411
Data columns (total 36 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CMPLNT_NUM                413412 non-null  int64  
 1   ADDR_PCT_CD               413412 non-null  int64  
 2   BORO_NM                   412927 non-null  object 
 3   CMPLNT_FR_DT              413412 non-null  object 
 4   CMPLNT_FR_TM              413412 non-null  object 
 5   CMPLNT_TO_DT              374308 non-null  object 
 6   CMPLNT_TO_TM              374433 non-null  object 
 7   CRM_ATPT_CPTD_CD          413412 non-null  object 
 8   HADEVELOPT                1570 non-null    object 
 9   HOUSING_PSA               30967 non-null   float64
 10  JURISDICTION_CODE         412949 non-null  float64
 11  JURIS_DESC                413412 non-null  object 
 12  KY_CD                     413412 non-null  int64  
 13  LAW_CAT_CD                413412 non-null  o

1it [00:01,  1.00s/it]


In [107]:
df_tmp.columns

Index(['CMPLNT_NUM', 'ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM',
       'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'CRM_ATPT_CPTD_CD', 'HADEVELOPT',
       'HOUSING_PSA', 'JURISDICTION_CODE', 'JURIS_DESC', 'KY_CD', 'LAW_CAT_CD',
       'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PATROL_BORO', 'PD_CD',
       'PD_DESC', 'PREM_TYP_DESC', 'RPT_DT', 'STATION_NAME', 'SUSP_AGE_GROUP',
       'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 'VIC_AGE_GROUP',
       'VIC_RACE', 'VIC_SEX', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude',
       'Longitude', 'Lat_Lon', 'New Georeferenced Column'],
      dtype='object')

In [108]:
df_tmp.shape

(413412, 36)

In [109]:
df_tmp.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,885776788,66,NaN,12/23/2020,19:50:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,18-24,BLACK,M,986633,167258,40.625769,-73.991417,"(40.62576896100006, -73.99141682199996)",POINT (-73.99141682199996 40.62576896100006)
1,350637195,77,NaN,12/21/2020,01:10:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,BLACK,M,1003606,185050,40.674583,-73.930222,"(40.67458330800008, -73.93022154099998)",POINT (-73.93022154099998 40.67458330800008)
2,347843168,43,BRONX,11/22/2020,22:00:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,BLACK,F,1020316,239179,40.823101,-73.869690,"(40.82310129900002, -73.86969046099993)",POINT (-73.86969046099993 40.82310129900002)
3,197941396,47,NaN,11/22/2020,09:50:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,BLACK,F,1026387,262634,40.887451,-73.847608,"(40.88745131300004, -73.84760778699997)",POINT (-73.84760778699997 40.88745131300004)
4,298404927,25,NaN,11/21/2020,15:38:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,18-24,BLACK HISPANIC,M,1003396,230824,40.800222,-73.930848,"(40.80022202900005, -73.93084834199995)",POINT (-73.93084834199995 40.80022202900005)


In [110]:
df_input=df_tmp[['CMPLNT_NUM','CMPLNT_FR_DT','CMPLNT_FR_TM','Latitude','Longitude']]

In [111]:
df_input.shape

(413412, 5)

In [112]:
df_input.dropna()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude
0,885776788,12/23/2020,19:50:00,40.625769,-73.991417
1,350637195,12/21/2020,01:10:00,40.674583,-73.930222
2,347843168,11/22/2020,22:00:00,40.823101,-73.869690
3,197941396,11/22/2020,09:50:00,40.887451,-73.847608
4,298404927,11/21/2020,15:38:00,40.800222,-73.930848
...,...,...,...,...,...
413407,947490808,01/04/2020,18:25:00,40.741341,-73.978393
413408,913801459,01/02/2020,20:30:00,40.688716,-73.826366
413409,927013283,01/02/2020,09:32:00,40.789947,-73.975354
413410,844073735,01/05/2020,12:55:00,40.882338,-73.891652


In [113]:
def counts_by_hexagon(df, resolution):
    """
    Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
    Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    
    Ex counts_by_hexagon(data, 9)
    """
    
    
    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["Latitude"], row["Longitude"], resolution), axis = 1)
    
   
   
    
   
    return df


In [114]:
%matplotlib inline
df_aggreg = counts_by_hexagon(df = df_input, resolution = 9)
# df_aggreg.sort_values(by = "value", ascending = False, inplace = True)
df_aggreg.head(5)
# hexmap = choropleth_map(df_aggreg = df_aggreg, with_legend = True)

<ipython-input-113-d4dfa957aace>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["Latitude"], row["Longitude"], resolution), axis = 1)


,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude,hex_id
0,885776788,12/23/2020,19:50:00,40.625769,-73.991417,892a1077163ffff
1,350637195,12/21/2020,01:10:00,40.674583,-73.930222,892a100d9cfffff
2,347843168,11/22/2020,22:00:00,40.823101,-73.869690,892a1001ac7ffff
3,197941396,11/22/2020,09:50:00,40.887451,-73.847608,892a1001477ffff
4,298404927,11/21/2020,15:38:00,40.800222,-73.930848,892a1008db3ffff


In [115]:
df_aggreg['str_datetime']=df_aggreg['CMPLNT_FR_DT']+' '+df_aggreg['CMPLNT_FR_TM']

<ipython-input-115-02f0bccb5098>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggreg['str_datetime']=df_aggreg['CMPLNT_FR_DT']+' '+df_aggreg['CMPLNT_FR_TM']


In [116]:
df_aggreg.drop(columns=['CMPLNT_FR_DT','CMPLNT_FR_TM'])

,CMPLNT_NUM,Latitude,Longitude,hex_id,str_datetime
0,885776788,40.625769,-73.991417,892a1077163ffff,12/23/2020 19:50:00
1,350637195,40.674583,-73.930222,892a100d9cfffff,12/21/2020 01:10:00
2,347843168,40.823101,-73.869690,892a1001ac7ffff,11/22/2020 22:00:00
3,197941396,40.887451,-73.847608,892a1001477ffff,11/22/2020 09:50:00
4,298404927,40.800222,-73.930848,892a1008db3ffff,11/21/2020 15:38:00
...,...,...,...,...,...
413407,947490808,40.741341,-73.978393,892a100d2bbffff,01/04/2020 18:25:00
413408,913801459,40.688716,-73.826366,892a100eb73ffff,01/02/2020 20:30:00
413409,927013283,40.789947,-73.975354,892a100882bffff,01/02/2020 09:32:00
413410,844073735,40.882338,-73.891652,892a10012c3ffff,01/05/2020 12:55:00


In [117]:
import datetime
import math

def get_julian_datetime(date):
    """
    Convert a datetime object into julian float.
    Args:
        date: datetime-object of date in question

    Returns: float - Julian calculated datetime.
    Raises: 
        TypeError : Incorrect parameter type
        ValueError: Date out of range of equation
    """

    # Ensure correct format
    if not isinstance(date, datetime.datetime):
        raise TypeError('Invalid type for parameter "date" - expecting datetime')
    elif date.year < 1801 or date.year > 2099:
        raise ValueError('Datetime must be between year 1801 and 2099')

    # Perform the calculation
    julian_datetime = 367 * date.year - int((7 * (date.year + int((date.month + 9) / 12.0))) / 4.0) + int(
        (275 * date.month) / 9.0) + date.day + 1721013.5 + (
                          date.hour + date.minute / 60.0 + date.second / math.pow(60,
                                                                                  2)) / 24.0 - 0.5 * math.copysign(
        1, 100 * date.year + date.month - 190002.5) + 0.5

    return julian_datetime

In [118]:
def julian_day(now):
    """
    1. Get current values for year, month, and day
    2. Same for time and make it a day fraction
    3. Calculate the julian day number via   https://en.wikipedia.org/wiki/Julian_day
    4. Add the day fraction to the julian day number

    """
    year = now.year
    month = now.month
    day = now.day
    day_fraction = now.hour + now.minute / 60.0 + now.second / 3600.0 / 24.0

    # The value 'march_on' will be 1 for January and February, and 0 for other months.
    march_on = math.floor((14 - month) / 12)
    year = year + 4800 - march_on
    # And 'month' will be 0 for March and 11 for February. 0 - 11 months
    month = month + 12 * march_on - 3

    y_quarter = math.floor(year / 4)
    jdn = day + math.floor((month * 153 + 2) / 5) + 365 * year + y_quarter

    julian = year < 1582 or year == (1582 and month < 10) or (month == 10 and day < 15)
    if julian:
        reform = 32083 # might need adjusting so needs a test
    else:
        reform = math.floor(year / 100) + math.floor(year / 400) + 32030.1875 # fudged this

    return jdn - reform + day_fraction

In [119]:
df_aggreg[['str_datetime']]

,str_datetime
0,12/23/2020 19:50:00
1,12/21/2020 01:10:00
2,11/22/2020 22:00:00
3,11/22/2020 09:50:00
4,11/21/2020 15:38:00
...,...
413407,01/04/2020 18:25:00
413408,01/02/2020 20:30:00
413409,01/02/2020 09:32:00
413410,01/05/2020 12:55:00


In [120]:
from datetime import datetime

In [121]:
df_aggreg['datetime'] = df_aggreg['str_datetime'].apply(lambda x:datetime.strptime(x,'%m/%d/%Y %H:%M:%S'))

<ipython-input-121-1bc5aad35d2f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggreg['datetime'] = df_aggreg['str_datetime'].apply(lambda x:datetime.strptime(x,'%m/%d/%Y %H:%M:%S'))


In [122]:
date_time_obj

datetime.datetime(2019, 9, 18, 1, 55, 19)

In [123]:
df_aggreg.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude,hex_id,str_datetime,datetime
0,885776788,12/23/2020,19:50:00,40.625769,-73.991417,892a1077163ffff,12/23/2020 19:50:00,2020-12-23 19:50:00
1,350637195,12/21/2020,01:10:00,40.674583,-73.930222,892a100d9cfffff,12/21/2020 01:10:00,2020-12-21 01:10:00
2,347843168,11/22/2020,22:00:00,40.823101,-73.869690,892a1001ac7ffff,11/22/2020 22:00:00,2020-11-22 22:00:00
3,197941396,11/22/2020,09:50:00,40.887451,-73.847608,892a1001477ffff,11/22/2020 09:50:00,2020-11-22 09:50:00
4,298404927,11/21/2020,15:38:00,40.800222,-73.930848,892a1008db3ffff,11/21/2020 15:38:00,2020-11-21 15:38:00


In [124]:
#pip install julian

In [125]:
import julian
import datetime

In [128]:
df_aggreg['julian_datetime'] = df_aggreg['datetime'].apply(lambda x:julian.to_jd(x + datetime.timedelta(hours=12), fmt='jd'))

<ipython-input-128-829fb4b98033>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggreg['julian_datetime'] = df_aggreg['datetime'].apply(lambda x:julian.to_jd(x + datetime.timedelta(hours=12), fmt='jd'))


In [129]:
df_aggreg.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude,hex_id,str_datetime,datetime,julian_datetime
0,885776788,12/23/2020,19:50:00,40.625769,-73.991417,892a1077163ffff,12/23/2020 19:50:00,2020-12-23 19:50:00,2.459208e+06
1,350637195,12/21/2020,01:10:00,40.674583,-73.930222,892a100d9cfffff,12/21/2020 01:10:00,2020-12-21 01:10:00,2.459205e+06
2,347843168,11/22/2020,22:00:00,40.823101,-73.869690,892a1001ac7ffff,11/22/2020 22:00:00,2020-11-22 22:00:00,2.459177e+06
3,197941396,11/22/2020,09:50:00,40.887451,-73.847608,892a1001477ffff,11/22/2020 09:50:00,2020-11-22 09:50:00,2.459176e+06
4,298404927,11/21/2020,15:38:00,40.800222,-73.930848,892a1008db3ffff,11/21/2020 15:38:00,2020-11-21 15:38:00,2.459176e+06


In [130]:
df_aggreg['julian_datetime'].value_counts()

2.458850e+06    155
2.458850e+06     82
2.458850e+06     80
2.459003e+06     59
2.459002e+06     58
               ... 
2.459007e+06      1
2.458953e+06      1
2.459185e+06      1
2.458984e+06      1
2.458927e+06      1
Name: julian_datetime, Length: 147815, dtype: int64

In [131]:
df_cleaned=df_aggreg.drop(columns=['CMPLNT_FR_DT','CMPLNT_FR_TM','str_datetime'])

In [133]:
df_cleaned.to_csv (r'D:\classes\ews_project\input_df.csv', index = None, header=True) 